## Data Ingestion Step

In [9]:
import os

In [10]:
os.chdir(r"C:\Users\Junior\Desktop\Predictive-Policing-Advisory")

## Trail-runs for enity_config

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path 
    unzip_dir: Path   

## Trial runs for Configuration file

In [12]:
#importing all project paths and modules necessary for project configurations 
from predictivePolicing.constants import  *
from predictivePolicing.utils.common import read_yaml, create_directories

In [13]:
#Updating the configuration file 
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
       
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file, 
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

## Trial runs for data_ingestion component

In [14]:
import os
from urllib import request
import zipfile
from predictivePolicing import logger
from predictivePolicing.utils.common import get_size

In [15]:
#Unpdate Components step
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_url, 
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with the following info: n\{headers}")
        else: 
            logger.info(f"File already exists: {get_size(Path(self.config.local_data_file))}")
        print("Downloding", self.config.source_url)
            
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts zip file into directory 
        Function returne none
        
        """
        
        unzip_path = self.config.unzip_dir
        # Print the actual path of the local data file and unzip directory
        print(f"Extracting file from: {self.config.local_data_file}")
        print(f"Unzip path: {unzip_path}")

        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

## Trial runs for data Ingestion Pipeline

In [16]:
#Update pipeline Step 
try: 
    config = ConfigurationManager()
    data_ingestion_config =config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-09-03 06:58:21,577: INFO: common: yaml_file: config\config.yaml loaded successfully]
[2025-09-03 06:58:21,583: INFO: common: yaml_file: params.yaml loaded successfully]
[2025-09-03 06:58:21,591: INFO: common: yaml_file: schema.yaml loaded successfully]
[2025-09-03 06:58:21,594: INFO: common: Created directory at: artifacts]
[2025-09-03 06:58:21,599: INFO: common: Created directory at: artifacts/data_ingestion]
[2025-09-03 06:58:24,909: INFO: 1819630040: artifacts/data_ingestion/data.zip download! with the following info: n\Connection: close
Content-Length: 1358075
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "3aa8cdebcb85a56cfcf99df4900fc165a78b05d0fadb330e4559ee09d943dce6"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: BE46:2125BB:44824:10307B:68B8497F
Accept-Ranges: bytes
Date: W